## Base de dados credit-data

In [1]:
import pandas as pd

df = pd.read_csv('credit-data.csv')

In [2]:
df.head(3)

,clientid,income,age,loan,default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0


Temos 5 colunas presentes no dataset fornecido, sendo três delas variáveis características (dados de entrada) e um delas uma variável-alvo (que queremos que o nosso modelo seja capaz de prever).

As variáveis características são:

    income       - O total de renda
    age          - Idade do usuário
    loan         - O empréstimo que o usuário fez

A variável-alvo é:

    Default    - um tipo *binário* que indica se o usuário pagou ou não: 
            0      - Usuário não pagou o empréstimo
            1      - Usuário pagou o empréstimo

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
clientid    2000 non-null int64
income      2000 non-null float64
age         1997 non-null float64
loan        2000 non-null float64
default     2000 non-null int64
dtypes: float64(3), int64(2)
memory usage: 78.2 KB


É notado que existem variáveis do tipo ``float64`` (números "decimais").

Já que todos os valores do nosso dataset são numéricos, nós não precisaremos fazer o a transformação do tipo "object" para o tipo númerico.

A função ``describe()`` gera várias informações sobre as variáveis numéricas que também podem ser úteis:

In [4]:
df.describe()

,clientid,income,age,loan,default
count,2000.000000,2000.000000,1997.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.807559,4444.369695,0.141500
std,577.494589,14326.327119,13.624469,3045.410024,0.348624
min,1.000000,20014.489470,-52.423280,1.377630,0.000000
25%,500.750000,32796.459717,28.990415,1939.708847,0.000000
50%,1000.500000,45789.117313,41.317159,3974.719419,0.000000
75%,1500.250000,57791.281668,52.587040,6432.410625,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000


#### Transformação 1: tratando dados negativos
Para tratar os dados negativos em nosso conjunto de dados, nós iremos alterar os valores negativos pela média de toda coluna.

In [5]:
# tirar a média de todos os valores da idade, e substituir os valores faltantes pela média.
df['age'].mean()

40.80755937840458

In [6]:
# porém devemos pegar a média dos valores positivos.
df['age'][df.age > 0].mean()

40.92770044906149

In [7]:
# Aqui alteramos as idades negativas para o valor da média acima.
df.loc[df.age < 0, 'age'] = 40.92
# ou
df[df['age'] < 0] = 40.92

In [8]:
# Nenhum valor :D
df.loc[df['age'] <= 0]

,clientid,income,age,loan,default


In [9]:
# ou podemos buscar pelos ID's (já que a base de dados é pequena)
df.loc[[15, 21, 26], "age"]

15    40.92
21    40.92
26    40.92
Name: age, dtype: float64

### Definição das features do nosso modelo
Para isso, criaremos a váriavel X que receberá os previsores do nosso modelo, e a variavel y que receberá a classe do nosso modelo.

Também retiraremos a coluna 'clientid' que não terá relevância no nosso modelo

In [10]:
# Definição das colunas que serão features (nota-se que a coluna 'clientid' não está presente)
features = ['income', 'age', 'loan']

# Definição da variável-alvo
target = ["default"]

# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = df[features].values
y = df[target].values

#### Encontrando dados faltantes (NaN)
Nesta parte será feito a procura dos dados faltantes no nosso dataset

In [11]:
# Com este comando descobrimos que há valores faltantes
pd.isnull(df['age']).value_counts()

False    1997
True        3
Name: age, dtype: int64

In [12]:
# ou os dados faltantes de todas colunas
faltantes = df.isnull().sum()
faltantes

clientid    0
income      0
age         3
loan        0
default     0
dtype: int64

In [13]:
# dados faltantes em %
faltantes_percentual = (df.isnull().sum() / len(df['clientid'])) * 100
faltantes_percentual

clientid    0.00
income      0.00
age         0.15
loan        0.00
default     0.00
dtype: float64

In [14]:
# com este comando podemos achar quem está com 'NaN'
df.loc[pd.isnull(df['age'])]

,clientid,income,age,loan,default
28,29.0,59417.805406,NaN,2082.625938,0.0
30,31.0,48528.852796,NaN,6155.784670,0.0
31,32.0,23526.302555,NaN,2862.010139,0.0


#### Transformação 2: Tratando dados faltantes (NaN)

Para tratar os dados faltantes em nosso conjunto de dados, iremos agora utilizar uma transformação pronta da biblioteca scikit-learn, chamada **SimpleImputer**.

Neste exemplo iremos simplesmente transformar todos os valores faltantes pela média.

In [15]:
import numpy as np
from sklearn.impute import SimpleImputer

In [16]:
# aqui é feita a estratégia do imputer, que no caso vamos mudar os valores NaN pela média da coluna
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer

SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
       verbose=0)

In [17]:
# aqui nós fazemos a mudança dos NaN apenas para a variavel 'imputer'
imputer = imputer.fit(X[:, 0:3])

In [18]:
# agora o 'previsores' recebe a transformação dos dados do 'imputer'
X[:, 0:3] = imputer.transform(X[:, 0:3])

###### Outra solução
Nós também podemos fazer o tratamento NaN 'a mão', sem importações, porém lembrando que altera o dataset original

In [19]:
# antes da transformação
df[df['age'].isnull() == True]

,clientid,income,age,loan,default
28,29.0,59417.805406,NaN,2082.625938,0.0
30,31.0,48528.852796,NaN,6155.784670,0.0
31,32.0,23526.302555,NaN,2862.010139,0.0


In [20]:
df['age'].fillna(df['age'].mean(), inplace = True)

In [21]:
# após a transformação
df[df['age'].isnull() == True].head()

,clientid,income,age,loan,default


#### Transformação 3: Escalonamento dos atributos (Padronização dos dados)
Nossos dados estão todos fora da mesma escala, e para isso nós temos que fazer a padronização deles

In [22]:
from sklearn.preprocessing import StandardScaler

In [23]:
scaler = StandardScaler()

In [24]:
X

array([[6.61559251e+04, 5.90170151e+01, 8.10653213e+03],
       [3.44151540e+04, 4.81171531e+01, 6.56474502e+03],
       [5.73171701e+04, 6.31080495e+01, 8.02095330e+03],
       ...,
       [4.43114493e+04, 2.80171669e+01, 5.52278669e+03],
       [4.37560566e+04, 6.39717958e+01, 1.62272260e+03],
       [6.94365796e+04, 5.61526170e+01, 7.37883360e+03]])

In [25]:
X = scaler.fit_transform(X)

In [26]:
X

array([[ 1.45393393,  1.36538093,  1.20281942],
       [-0.76217555,  0.5426602 ,  0.69642695],
       [ 0.83682073,  1.67417189,  1.17471147],
       ...,
       [-0.07122592, -0.97448519,  0.35420081],
       [-0.11000289,  1.73936739, -0.92675625],
       [ 1.682986  ,  1.14917639,  0.96381038]])

<hr>

### Treinando os modelos de classificação
Finalizado o pré-processamento, já temos o conjunto de dados no formato necessário para o treinamento do nosso modelo

### Importações dos algoritmos de ML 

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
# Redes neurais com keras
import keras
from keras.models import Sequential
from keras.layers import Dense

#### Separando o dataset em um conjunto de treino e um conjunto de teste
Iremos separar o dataset fornecido em dois grupos: um para treinar nosso modelo, e outro para testarmos o resultado através de um teste cego.

In [28]:
# Separação dos dados em um conjunto de treino e um conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

<hr>

### Naive Bayes 
O algoritmo Naive Bayes é um algoritmo simples de classificação, que utiliza dados históricos para prever a classificação de um novo dado. Ele funciona calculando a probabilidade de um evento ocorrer dado que outro evento já ocorreu

In [29]:
nb = GaussianNB()

In [30]:
# Parte do treinamento
nb.fit(X_train, y_train)
# Parte do teste, predict serve para testar
y_pred_nb = nb.predict(X_test)

C:\Users\Lucas\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [31]:
# Acurácia alcançada pelo algortimo Naive Bayes
print("Acurácia: {}%".format(100*round(accuracy_score(y_test, y_pred_nb), 2)))

Acurácia: 94.0%


In [32]:
print(classification_report(y_test, y_pred_nb))

              precision    recall  f1-score   support

         0.0       0.95      0.98      0.97       436
         1.0       0.84      0.64      0.73        64

   micro avg       0.94      0.94      0.94       500
   macro avg       0.89      0.81      0.85       500
weighted avg       0.93      0.94      0.93       500



In [33]:
# matriz de confusão de acertos
confusion_matrix(y_test, y_pred_nb)

array([[428,   8],
       [ 23,  41]], dtype=int64)

<hr>

### Decision Tree
O algoritmo Decision Tree são modelos estatísticos que utilizam um treinamento supervisionado para a classificação e previsão de dados, Estes modelos utilizam a estratégia de dividir para conquistar: um problema complexo é decomposto em sub-problemas mais simples e recursivamente esta técnica é aplicada a cada sub-problema

In [34]:
dt = DecisionTreeClassifier(criterion = 'entropy')

In [35]:
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [36]:
y_pred_dt = dt.predict(X_test)

In [37]:
# Acurácia alcançada pelo algortimo Decision Tree
print("Acurácia: {}%".format(100*round(accuracy_score(y_test, y_pred_dt), 2)))

Acurácia: 98.0%


In [38]:
print(classification_report(y_test, y_pred_dt))

              precision    recall  f1-score   support

         0.0       1.00      0.99      0.99       436
         1.0       0.91      0.97      0.94        64

   micro avg       0.98      0.98      0.98       500
   macro avg       0.95      0.98      0.97       500
weighted avg       0.98      0.98      0.98       500



In [39]:
# matriz de confusão de acertos
confusion_matrix(y_test, y_pred_dt)

array([[430,   6],
       [  2,  62]], dtype=int64)

<hr>

### Random Forest
O algoritmo Random Forest cria uma floresta de um modo aleatório, criando várias árvores de decisão e as combinando,cada árvore tenta estimar uma classificação e isso é chamado como “voto”, assim, para obter uma predição com maior acurácia e mais estável.

In [40]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy', random_state=0)

In [41]:
rf.fit(X_train, y_train)

C:\Users\Lucas\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [42]:
y_pred_rf = rf.predict(X_test)

In [43]:
# Acurácia alcançada pelo algortimo Random Forest
print("Acurácia: {}%".format(100*round(accuracy_score(y_test, y_pred_rf), 2)))

Acurácia: 98.0%


In [44]:
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99       436
         1.0       0.95      0.89      0.92        64

   micro avg       0.98      0.98      0.98       500
   macro avg       0.97      0.94      0.95       500
weighted avg       0.98      0.98      0.98       500



In [45]:
# matriz de confusão de acertos
confusion_matrix(y_test, y_pred_rf)

array([[433,   3],
       [  7,  57]], dtype=int64)

<hr>

### kNN (Vizinhos mais próximos)
O algoritmo KNN ou k-vizinhos mais próximos é um algoritmo bem simples de machine learning. Ele usa algum tipo de medida de similaridade para dizer em qual classe o novo dado se classifica, neste caso utilizaremos 5 vizinhos mais próximos.

In [46]:
knn = KNeighborsClassifier(n_neighbors=5, metric = 'minkowski', p = 2)

In [47]:
knn.fit(X_train, y_train)

C:\Users\Lucas\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [48]:
y_pred_kn = knn.predict(X_test)

In [49]:
# Acurácia alcançada pelo algortimo kNN
print("Acurácia: {}%".format(100*round(accuracy_score(y_test, y_pred_kn), 2)))

Acurácia: 99.0%


In [50]:
print(classification_report(y_test, y_pred_kn))

              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99       436
         1.0       0.94      0.95      0.95        64

   micro avg       0.99      0.99      0.99       500
   macro avg       0.97      0.97      0.97       500
weighted avg       0.99      0.99      0.99       500



In [51]:
# matriz de confusão de acertos
confusion_matrix(y_test, y_pred_kn)

array([[432,   4],
       [  3,  61]], dtype=int64)

<hr>

### Regressão Logistica
O algoritmo de regressão logística é usada onde uma saída discreta é esperada, (ex. Prever se um usuário é um bom ou mal pagador).Normalmente, a regressão logística usa alguma função para espremer valores para um determinado intervalo.

In [52]:
rl = LogisticRegression(random_state=0)

In [53]:
rl.fit(X_train, y_train)

C:\Users\Lucas\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Lucas\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [54]:
y_pred_rl = rl.predict(X_test)

In [55]:
# Acurácia alcançada pelo algortimo kNN
print("Acurácia: {}%".format(100*round(accuracy_score(y_test, y_pred_rl), 2)))

Acurácia: 94.0%


In [56]:
print(classification_report(y_test, y_pred_rl))

              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97       436
         1.0       0.78      0.78      0.78        64

   micro avg       0.94      0.94      0.94       500
   macro avg       0.87      0.87      0.87       500
weighted avg       0.94      0.94      0.94       500



In [57]:
# matriz de confusão de acertos
confusion_matrix(y_test, y_pred_rl)

array([[422,  14],
       [ 14,  50]], dtype=int64)

<hr>

### SVM (Máquinas de vetores de suporte)
O algoritmo de SVM separa os pontos de dados usando uma linha. Esta linha é escolhida de tal forma que será mais importante dos pontos de dados mais próximos em 2 categorias.

In [58]:
svm = SVC(kernel = 'rbf', C = 2.0, random_state= 0)

In [59]:
svm.fit(X_train, y_train)

C:\Users\Lucas\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Lucas\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=2.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=0,
  shrinking=True, tol=0.001, verbose=False)

In [60]:
y_pred_svm = svm.predict(X_test)

In [61]:
# Acurácia alcançada pelo algortimo kNN
print("Acurácia: {}%".format(100*round(accuracy_score(y_test, y_pred_svm), 2)))

Acurácia: 99.0%


In [62]:
print(classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99       436
         1.0       0.97      0.94      0.95        64

   micro avg       0.99      0.99      0.99       500
   macro avg       0.98      0.97      0.97       500
weighted avg       0.99      0.99      0.99       500



In [63]:
# matriz de confusão de acertos
confusion_matrix(y_test, y_pred_svm)

array([[434,   2],
       [  4,  60]], dtype=int64)

<hr>

### Redes Neurais
O objetivo do algoritmo de Redes neurais é imitar o sistema nervoso de humanos no processo de aprendizagem, ela é inspirada nas redes neurais biológicas

Redes Neurais com a biblioteca ``sklearn``

In [64]:
rn = MLPClassifier(verbose=True, max_iter=100, tol = 0.000010, solver = 'adam', hidden_layer_sizes= (100), activation='relu')

In [65]:
rn.fit(X_train, y_train)

C:\Users\Lucas\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.82643083
Iteration 2, loss = 0.74027504
Iteration 3, loss = 0.66385755
Iteration 4, loss = 0.59789300
Iteration 5, loss = 0.54147403
Iteration 6, loss = 0.49250850
Iteration 7, loss = 0.45048935
Iteration 8, loss = 0.41415965
Iteration 9, loss = 0.38260762
Iteration 10, loss = 0.35499412
Iteration 11, loss = 0.33118433
Iteration 12, loss = 0.30994897
Iteration 13, loss = 0.29135575
Iteration 14, loss = 0.27483603
Iteration 15, loss = 0.26005814
Iteration 16, loss = 0.24673154
Iteration 17, loss = 0.23477151
Iteration 18, loss = 0.22394420
Iteration 19, loss = 0.21407142
Iteration 20, loss = 0.20516912
Iteration 21, loss = 0.19704154
Iteration 22, loss = 0.18947024
Iteration 23, loss = 0.18265776
Iteration 24, loss = 0.17627689
Iteration 25, loss = 0.17037564
Iteration 26, loss = 0.16492699
Iteration 27, loss = 0.15990927
Iteration 28, loss = 0.15515962
Iteration 29, loss = 0.15076590
Iteration 30, loss = 0.14670879
Iteration 31, loss = 0.14284676
Iteration 32, los

C:\Users\Lucas\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=100, learning_rate='constant',
       learning_rate_init=0.001, max_iter=100, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=1e-05,
       validation_fraction=0.1, verbose=True, warm_start=False)

In [66]:
y_pred_rn = rn.predict(X_test)

In [67]:
# Acurácia alcançada pelo algortimo kNN
print("Acurácia: {}%".format(100*round(accuracy_score(y_test, y_pred_rn), 2)))

Acurácia: 98.0%


In [68]:
print(classification_report(y_test, y_pred_rn))

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99       436
         1.0       0.93      0.88      0.90        64

   micro avg       0.98      0.98      0.98       500
   macro avg       0.96      0.93      0.94       500
weighted avg       0.98      0.98      0.98       500



In [69]:
# matriz de confusão de acertos
confusion_matrix(y_test, y_pred_rn)

array([[432,   4],
       [  8,  56]], dtype=int64)

Redes Neurais com a biblioteca ``Keras``

In [70]:
rn_keras = Sequential()
# neurônios, ativação, neurônios iniciais 
rn_keras.add(Dense(units = 2, activation = 'relu', input_dim = 3))
# Camada 2 (oculta)
rn_keras.add(Dense(units = 2, activation = 'relu'))
# Camada de saida
rn_keras.add(Dense(units = 1, activation = 'sigmoid'))
# Ajustes finais (Adam, calculo do erro (binário), métricas seria a %)
rn_keras.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [71]:
# batch_size (a cada 10 registros é atualizado os pesos)
# nb_epochs (epócas: executar 100 vezes)
rn_keras.fit(X_train, y_train, batch_size = 10, epochs= 100)

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
150/150 [==============================] - 0s 1ms/step - loss: 0.7177 - accuracy: 0.8540
Epoch 2/50
150/150 [==============================] - 0s 1ms/step - loss: 0.6211 - accuracy: 0.8540
Epoch 3/50
150/150 [==============================] - 0s 1ms/step - loss: 0.5305 - accuracy: 0.8540
Epoch 4/50
150/150 [==============================] - 0s 1ms/step - loss: 0.45

In [72]:
y_pred_rn_keras = rn_keras.predict(X_test)
y_pred_rn_keras = (y_pred_rn_keras > 0.5)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [73]:
# Acurácia alcançada pelo algortimo kNN
print("Acurácia: {}%".format(100*round(accuracy_score(y_test, y_pred_rn_keras), 2)))

Acurácia: 95.0%


In [74]:
print(classification_report(y_test, y_pred_rn_keras))

              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97       436
         1.0       0.80      0.80      0.80        64

   micro avg       0.95      0.95      0.95       500
   macro avg       0.88      0.88      0.88       500
weighted avg       0.95      0.95      0.95       500

